### Clustering locations into groups of similar coordinates

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth
# in-situ coordinates
insitu_coords = pd.read_csv('geometry.txt',sep=' ')

y = insitu_coords[['xcoord','ycoord','zcoord']].values
bw = estimate_bandwidth(y, quantile=.01)
ms = MeanShift(bandwidth=bw, bin_seeding=True, min_bin_freq=5)
ms.fit(y)
# store coordinates for the centers of the clusters
cluster_centers = ms.cluster_centers_
np.save('cluster_centers.npy',cluster_centers)

# store what cluster the insitu coordinates belong to
cluster_label = list(ms.labels_)
with open('cluster_label.txt','w') as f:
    f.write('\n'.join(map(str,cluster_label)))

### Feature selection

In [2]:
import numpy as np
import pandas as pd
import scipy
from sklearn import linear_model
from skfeature.function.sparse_learning_based import MCFS
from skfeature.utility.construct_W import construct_W

def mcfs(X, n_selected_features):
    """
    This function implements unsupervised feature selection for multi-cluster data.

    Input
    -----
    X: {numpy array}, shape (n_samples, n_features)
        input data
    n_selected_features: {int}
        number of features to select
    kwargs: {dictionary}
        W: {sparse matrix}, shape (n_samples, n_samples)
            affinity matrix
        n_clusters: {int}
            number of clusters (default is 5)

    Output
    ------
    W: {numpy array}, shape(n_features, n_clusters)
        feature weight matrix

    Reference
    ---------
    Cai, Deng et al. "Unsupervised Feature Selection for Multi-Cluster Data." KDD 2010.
    """

    # use the default affinity matrix
    kwargs = {"metric": 'euclidean', "neighborMode": "knn", "weightMode": 'binary', "k": 45, 't': 1}
    if 'W' not in kwargs:
        W = construct_W(X, **kwargs)
    else:
        W = kwargs['W']
    # default number of clusters is 5
    if 'n_clusters' not in kwargs:
        n_clusters = 5
    else:
        n_clusters = kwargs['n_clusters']

    # solve the generalized eigen-decomposition problem and get the top K
    # eigen-vectors with respect to the smallest eigenvalues
    W = W.toarray()
    W = (W + W.T) / 2
    W_norm = np.diag(np.sqrt(1 / W.sum(1)))
    W = np.dot(W_norm, np.dot(W, W_norm))
    WT = W.T
    W[W < WT] = WT[W < WT]
    eigen_value, ul = scipy.linalg.eigh(a=W)
    Y = np.dot(W_norm, ul[:, -1*n_clusters-1:-1])

    # solve K L1-regularized regression problem using LARs algorithm with cardinality constraint being d
    n_sample, n_feature = X.shape
    W = np.zeros((n_feature, n_clusters))
    for i in range(n_clusters):
        clf = linear_model.Lars(n_nonzero_coefs=n_selected_features)
        clf.fit(X, Y[:, i])
        W[:, i] = clf.coef_
    return W


# Get marker genes from in situ data
insitu_df = pd.read_csv('bdtnp.txt',sep='\t')
marker_genes = list(insitu_df.columns)

# Get expression of marker genes for cells
cell_df = pd.read_csv('dge_normalized.txt',sep='\t')
all_genes = list(cell_df.index)  
cell_df = cell_df.T
cell_df.columns = all_genes
cell_df = cell_df[marker_genes]

# rank features using MCFS feature selection method
with open('fs_mcfs_20.txt', 'w') as f:
    score = mcfs(cell_df.values,20)
    idx = MCFS.feature_ranking(score)
    ls = [marker_genes[e] for e in idx][:20]
    f.write('\n'.join(ls))

### Location prediction for cells

In [3]:
from sklearn.neighbors import KNeighborsClassifier
import scipy

# this function returns MCC score for true_labels and pred_labels vectors
def get_mcc(true_labels, pred_labels):
    TP = np.sum(np.logical_and(pred_labels == 1, true_labels == 1))
    TN = np.sum(np.logical_and(pred_labels == 0, true_labels == 0))
    FP = np.sum(np.logical_and(pred_labels == 1, true_labels == 0))
    FN = np.sum(np.logical_and(pred_labels == 0, true_labels == 1))
    mcc = (TP * TN) - (FP * FN)
    denom = np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    if denom==0:
        return 0
    return mcc / denom

# turn the probability to the clusters of the cell to the weighted coordinates of the cluster centers
def get_xyz(x):
    ret = []
    for i in range(len(x)):
        v = np.average(cluster_centers, axis=0, weights=x[i,:])
        ret += [ v ]
    return np.array(ret)

# in-situ binary expression
insitu_bin = pd.read_csv('binarized_bdtnp.csv')
# in-situ coordinates
insitu_coords = pd.read_csv('geometry.txt',sep=' ')

# cell binary expression
cell_bin = pd.read_csv('dge_binarized_distMap.csv')
all_genes = list(cell_bin.index)  
cell_bin = cell_bin.T
cell_bin.columns = all_genes

labels = [int(e) for e in open('cluster_label.txt')]
labels = np.array(labels)
cluster_centers = np.load('cluster_centers.npy')


genes = [e.strip() for e in open('fs_mcfs_20.txt') if e.strip()]
clf = KNeighborsClassifier(n_neighbors=70, leaf_size=50, metric='matching')
# learn a multi-class classifier
clf.fit(insitu_bin[genes].values, labels)
# predict probability of cells to the clusters
preds = clf.predict_proba(cell_bin[genes].values)
# turn the probabilities to average weighted coordinates
preds = get_xyz(preds)
# distance to the in situ coordinates of the predicted coordinates for cells
preds_to_insitu_coords = scipy.spatial.distance.cdist(preds,insitu_coords[['xcoord','ycoord','zcoord']].values)

# choose 10 best locations
TOP = 10
# store indices of locations for all cells
indices = []
for i in range(len(preds_to_insitu_coords)):
#     get the index of the location whose the smallest to insitu coords
    best_idx = preds_to_insitu_coords[i].argmin()
#     get coordinates for the location
    coord = insitu_coords[ insitu_coords.index==best_idx ][['xcoord','ycoord','zcoord']].values
#     compared the coord with all the coords    
    dist = scipy.spatial.distance.cdist(coord, insitu_coords[['xcoord','ycoord','zcoord']].values)[0]
#     get the locations with shortest Euclidean distance
    top_idx = dist.argsort()[:TOP]
#     +1 as the location starts from 1
    top_idx = [e+1 for e in top_idx]
    indices += [ top_idx ]

# submission file
dout = '20genes.csv' 
with open(dout,'w') as file_result:
    tmp = ['']
    for i in range(len(genes)):
        tmp += [ genes[i] ]
        if (i+1)%10==0:
            file_result.write(','.join(tmp) + '\n')
            tmp = ['']
    for i in range(len(indices)):
        tmp = [i+1] + indices[i]
        file_result.write(','.join(map(str,tmp)) + '\n')